# Knowledge Base

## Auswahl der Sprachmodelle 

### Llama 3.3 (8B Version)
- **Leistung**: Starke Performance für RAG-Anwendungen mit guter Kontextverarbeitung
- **Größe**: Mit 8B Parametern effizient auf Consumer-Hardware lauffähig
- **Lizenz**: Permissive Lizenz ermöglicht kommerzielle Nutzung
- **Aktualität**: Neues Modell mit modernem Trainingsdatenset und verbesserter Instruction-Following-Fähigkeit

### Mistral 7B Instruct
- **Effizienz**: Ausgezeichnetes Leistungs-Größen-Verhältnis
- **Spezialisierung**: Optimiert für Instruction-Following und Kontextverständnis
- **Architektur**: Gruppenweise Rotation der Aufmerksamkeit für verbesserte Verarbeitung langer Dokumente
- **Community-Support**: Breite Nutzerbasis und dokumentierte Anwendungsfälle für RAG

### Phi-4 (Mini)
- **Ressourcenschonung**: Kleines Modell (3.8B) für Systeme mit begrenzten Ressourcen
- **Effizienz**: Hervorragende Leistung trotz geringer Größe
- **Antwortqualität**: Gute Formulierungsfähigkeit bei unternehmensbezogenen Inhalten
- **Kompatibilität**: Geringer VRAM-Bedarf macht es auf verschiedenen Systemen einsetzbar

Dieser Mix bietet eine gute Balance zwischen Performance, Ressourcenbedarf und verschiedenen Architekturen für einen aussagekräftigen Vergleich.



## Einrichtung der Knowledge Base

### Verzeichnisstruktur + Imports

In [2]:
import os
import shutil
from pathlib import Path
import fitz
import re
from typing import List
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain.schema import Document

BASE_DIR = Path("knowledge-base")
IMPORT_DIR = BASE_DIR / "import"
PROCESSED_DIR = BASE_DIR / "processed"
INPUT_DIR = BASE_DIR / "embeddings-ready"
VECTOR_DB_DIR = BASE_DIR / "vector-stores"
VECTOR_DB_DIR.mkdir(exist_ok=True, parents=True)

for dir_path in [BASE_DIR, IMPORT_DIR, PROCESSED_DIR, INPUT_DIR]:
    dir_path.mkdir(exist_ok=True, parents=True)

### Dokumenten-Import

In [3]:
   
def list_pdf_files(directory):
    return [f for f in directory.glob("*.pdf")]
  
pdf_files = list_pdf_files(IMPORT_DIR)

print(f"PDF-Dateien: {len(pdf_files)}")
for pdf in pdf_files[:5]:  # Zeige die ersten 5 Dateien
    print(f" - {pdf.name}")

def process_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text_content = []
    
    for page_num in range(len(document)):
        page = document[page_num]
        text = page.get_text()
        
        # Grundlegende Textbereinigung
        text = re.sub(r'\s+', ' ', text)  # Mehrfach-Leerzeichen entfernen
        text = text.strip()
        
        if text:
            text_content.append(f"--- Seite {page_num + 1} ---\n{text}")
    
    document.close()
    
    # Gesamten Text zusammenführen
    processed_text = "\n\n".join(text_content)
    
    # Weitere Bereinigungen für bessere LLM-Verarbeitung
    processed_text = re.sub(r'([.!?])\s*(\w)', r'\1\n\2', processed_text)  # Satzenden mit Zeilenumbrüchen
    
    return processed_text

# 4. Dateien verarbeiten und verschieben
processed_files = []

for pdf_path in pdf_files:
    print(f"Verarbeite: {pdf_path.name}")
    
    # Text extrahieren und aufbereiten
    processed_text = process_pdf(pdf_path)
    
    # Ausgabedatei im embeddings-ready Verzeichnis erstellen
    output_file = INPUT_DIR / f"{pdf_path.stem}.txt"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(processed_text)
    
    # Originaldatei in processed-Verzeichnis verschieben
    target_path = PROCESSED_DIR / pdf_path.name
    shutil.move(pdf_path, target_path)
    
    processed_files.append({
        "original_file": pdf_path.name,
        "processed_file": output_file.name,
        "size_kb": round(output_file.stat().st_size / 1024, 2)
    })

PDF-Dateien: 0


## Implementierung der Content Embeddings 

### Leitfragen zur Bewertung

### Ausgewählte Embeddings

### Ergebnis

In [4]:
EMBEDDING_MODELS = {
    "Word-Level": "sentence-transformers/all-MiniLM-L6-v2",
    "Sentence-Level": "sentence-transformers/all-mpnet-base-v2",
    "Document-Level": "intfloat/multilingual-e5-large"
}

# Fixed values
CONTEXT_WINDOW = 8192
TOKEN_LIMIT = 4096

# Document loading function (unchanged)
def load_documents(directory: Path) -> List[Document]:
    """Lädt alle Textdateien aus dem angegebenen Verzeichnis."""
    documents = []
    
    for file_path in directory.glob("*.txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
        
        doc = Document(
            page_content=text,
            metadata={"source": file_path.name}
        )
        documents.append(doc)
    
    print(f"Geladen: {len(documents)} Dokumente")
    return documents

# Text-Splitter functions (only keeping word-level for now)
def prepare_word_level_chunks(documents: List[Document]) -> List[Document]:
    """Teilt Dokumente in überlappende Wortgruppen auf."""
    # Use fixed values divided by 2 as in original logic
    max_chunk_size = min(CONTEXT_WINDOW // 2, TOKEN_LIMIT)
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chunk_size,
        chunk_overlap=50,  # Fixed overlap
        length_function=len
    )
    return text_splitter.split_documents(documents)
  
def create_vector_database(documents: List[Document], embedding_type: str, 
                          model_name: str, model_path: str, vector_db_dir: Path):
    """Erstellt eine Vektordatenbank mit den angegebenen Embeddings."""
    print(f"\n--- Erstelle Vektordatenbank: {embedding_type} mit {model_name} ---")
    
    # Embeddings initialisieren
    embeddings = HuggingFaceEmbeddings(model_name=model_path)
    
    # Vektordatenbank-Pfad definieren
    db_path = vector_db_dir / f"{embedding_type}_{model_name.replace('/', '_')}"
    
    # Vektordatenbank erstellen
    db = Chroma.from_documents(
        documents, 
        embeddings, 
        persist_directory=str(db_path)
    )
    
    print(f"Vektordatenbank wurde in {db_path} gespeichert.")
    return db
  
documents = load_documents(INPUT_DIR)
if not documents:
    print("Keine Dokumente gefunden!")
else:
  # Dokumente in Chunks aufteilen (einmalig)
  word_chunks = prepare_word_level_chunks(documents)
  print(f"Dokumente in {len(word_chunks)} Chunks aufgeteilt.")
  
  # Für jedes Embedding-Modell eine Vektordatenbank erstellen
  for embedding_type, model_path in EMBEDDING_MODELS.items():
      model_name = model_path.split("/")[-1]
      
      # Vektordatenbank erstellen
      create_vector_database(
          word_chunks, embedding_type, model_name, model_path, VECTOR_DB_DIR
      )

Geladen: 10 Dokumente
Dokumente in 222 Chunks aufgeteilt.

--- Erstelle Vektordatenbank: Word-Level mit all-MiniLM-L6-v2 ---


c:\Projects\campus\mle\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vektordatenbank wurde in knowledge-base\vector-stores\Word-Level_all-MiniLM-L6-v2 gespeichert.

--- Erstelle Vektordatenbank: Sentence-Level mit all-mpnet-base-v2 ---
Vektordatenbank wurde in knowledge-base\vector-stores\Sentence-Level_all-mpnet-base-v2 gespeichert.

--- Erstelle Vektordatenbank: Document-Level mit multilingual-e5-large ---
Vektordatenbank wurde in knowledge-base\vector-stores\Document-Level_multilingual-e5-large gespeichert.


## Implementierung des Chunking 

In [5]:
SELECTED_EMBEDDING = "Document-Level"
EMBEDDING_MODEL = EMBEDDING_MODELS[SELECTED_EMBEDDING]

CHUNKING_METHODS = {
    "fixed_size": RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=100
    ),
    
    "sentence": RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ". ", "! ", "? ", ";", ":"],
        chunk_size=1000,
        chunk_overlap=0
    ),
    
    "paragraph": RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n"],
        chunk_size=2000,
        chunk_overlap=50
    )
}

def create_vector_database_with_chunking(documents: List[Document], method_name: str, 
                                        splitter, vector_db_dir: Path):
    """Erstellt eine Vektordatenbank mit der angegebenen Chunking-Methode."""
    print(f"\n--- Erstelle Vektordatenbank mit Chunking-Methode: {method_name} ---")
    
    # Dokumente chunken
    chunks = []
    try:
        for doc in documents:
            if hasattr(splitter, 'split_documents'):
                # Für TextSplitter, die direkt Dokumente unterstützen
                doc_chunks = splitter.split_documents([doc])
            else:
                # Für TextSplitter, die nur Text unterstützen
                text_chunks = splitter.split_text(doc.page_content)
                doc_chunks = [
                    Document(page_content=chunk, metadata=doc.metadata)
                    for chunk in text_chunks
                ]
            chunks.extend(doc_chunks)
    except Exception as e:
        print(f"Fehler beim Chunking mit {method_name}: {e}")
        return None
    
    print(f"Chunks erstellt: {len(chunks)}")
    
    if not chunks:
        print("Keine Chunks erzeugt!")
        return None
    
    # Beispiel-Chunks anzeigen
    print("\nBeispiel-Chunks:")
    for i in range(min(2, len(chunks))):
        print(f"Chunk {i+1}, Größe: {len(chunks[i].page_content)} Zeichen")
        print(f"Quelle: {chunks[i].metadata.get('source', 'Unbekannt')}")
        print(f"Inhalt: {chunks[i].page_content[:150]}...\n")
    
    # Embeddings erstellen und in Vector Store speichern
    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
    
    db_path = vector_db_dir / f"chunking_{method_name}"
    
    db = Chroma.from_documents(
        chunks, 
        embeddings, 
        persist_directory=str(db_path)
    )
    
    print(f"Vektordatenbank wurde in {db_path} gespeichert.")
    return db
  
print(f"Chunking-Methoden mit {SELECTED_EMBEDDING} Embedding-Modell")
  
# Dokumente laden
documents = load_documents(INPUT_DIR)
if not documents:
    print("Keine Dokumente gefunden!")
else:
  # Chunking-Methoden testen
  for method_name, splitter in CHUNKING_METHODS.items():
      print(f"\n=== Chunking-Methode: {method_name} ===")
      create_vector_database_with_chunking(documents, method_name, splitter, VECTOR_DB_DIR)

Chunking-Methoden mit Document-Level Embedding-Modell
Geladen: 10 Dokumente

=== Chunking-Methode: fixed_size ===

--- Erstelle Vektordatenbank mit Chunking-Methode: fixed_size ---
Chunks erstellt: 759

Beispiel-Chunks:
Chunk 1, Größe: 747 Zeichen
Quelle: 1-Challenges_and_Reliability_of_Predictive_Maintenance.txt
Inhalt: --- Seite 1 ---
See discussions, stats, and author profiles for this publication at: https://www.
researchgate.
net/publication/331951459 Challenges a...

Chunk 2, Größe: 952 Zeichen
Quelle: 1-Challenges_and_Reliability_of_Predictive_Maintenance.txt
Inhalt: --- Seite 3 ---
Declaration of Authorship I, Vincent F.
A.
Meyer zu Wickern, declare that this thesis titled, ‘Challenges and Reliability of Predictiv...

Vektordatenbank wurde in knowledge-base\vector-stores\chunking_fixed_size gespeichert.

=== Chunking-Methode: sentence ===

--- Erstelle Vektordatenbank mit Chunking-Methode: sentence ---
Chunks erstellt: 745

Beispiel-Chunks:
Chunk 1, Größe: 747 Zeichen
Quelle: 1

## Test

In [10]:
def ask_ollama(query, vector_db_dir=VECTOR_DB_DIR):
    """RAG mit Phi-4-mini über Ollama"""
    import requests
    
    # DB laden und abfragen
    embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
    db = Chroma(persist_directory=str(vector_db_dir / "chunking_sentence"), embedding_function=embeddings)
    results = db.similarity_search(query, k=3)
    context = "\n\n".join([doc.page_content for doc in results])
    
    # Prompt erstellen
    prompt = f"""Basierend auf folgendem Kontext, beantworte die Frage.

Kontext:
{context}

Frage: {query}

Antwort:"""
    
    # Ollama API anfragen
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": "phi4-mini", "prompt": prompt, "stream": False}
        )
        return response.json().get("response", "") if response.status_code == 200 else f"Fehler: {response.status_code}"
    except Exception as e:
        return f"Fehler: {str(e)}"

# Beispiel
query = "What is an F1-Score in anomaly detection?"
print(ask_ollama(query))

An F1-Score in anomaly detection (also known as precision-recall curve AUC) combines both Precision and Recall into one metric. It helps assess how well-balanced two measures are for a particular classification system, particularly useful when the class distribution may be imbalanced.

Precision is defined as true positive rate or sensitivity: TPR = TP/(TP+FN). In this context it refers to correctly identified anomalies (True Positives) divided by all predicted positives. The F1-Score combines Precision and Recall with equal weighting into one metric.
Recall, on the other hand, measures how many actual anomaly events are detected as such; in other words TPR = TP/(TP+FN). In this context it refers to correctly identified anomalies (True Positives) divided by all positive samples. The F1-Score combines Precision and Recall with equal weighting into one metric.
An adjusted version of the F1 score accounts for both false positives, denoted as FP's, and missed detections FN' 's; thus allowi